# Baseline RAG example

This is a simple example of a baseline RAG application which purpose is to answer questions about the fantasy series [Malazan Universe](https://malazan.fandom.com/wiki/Malazan_Wiki) created by Steven Erikson and Ian C. Esslemont.

First the example will show each step of a baseline RAG pipeline including **Indexing**, **Retrieval** and **Generation**. This is done in order to show the architecture without the abstraction provided by frameworks like LlamaIndex and LangChain.
Then a more "normal" example will be shown using LlamaIndex.

As a vector database, we will use [ChromaDB](https://docs.trychroma.com/), but this can easily be exchanged with other databases.

In this example, we will use the following technologies

- OpenAI API
- ChromaDB
- LlamaIndex


### Setup libraries and environment


In [1]:
#%pip install chromadb llama-index-vector-stores-chroma

In [2]:
import os

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb import Settings
from IPython.display import Markdown, display
from llama_index.core import PromptTemplate, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from openai import OpenAI, AzureOpenAI

import importlib
import util

# Suppose you made some changes to util.py

#importlib.reload(util.helpers)
from util.helpers import create_and_save_md_files, get_malazan_pages, get_theoffice_pages

[nltk_data] Downloading package punkt_tab to
[nltk_data]     c:\Users\jach\AppData\Local\miniconda3\envs\advanced-
[nltk_data]     rag-env\Lib\site-
[nltk_data]     packages\llama_index\core\_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


### Environment variables

For this example you need to use an OpenAI API key. Go to [your API keys](https://platform.openai.com/api-keys) in the OpenAI console to generate one.

Then add the following to a `.env` file in the root of the project.

```
OPENAI_API_KEY=<YOUR_KEY_HERE>
```


In [12]:
from dotenv import load_dotenv

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai_client = AzureOpenAI(
    api_key=OPENAI_API_KEY,  
    api_version="2024-05-01-preview", # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [15]:

#openai_client = AzureOpenAI(api_key=OPENAI_API_KEY)
#openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#    api_key=OPENAI_API_KEY,
#    model_name="text-embedding-3-small"
#)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-3-small", #"text-embedding-ada-002",
    api_type="azure"
)

chroma_client = chromadb.PersistentClient(
    path="./data/seges-gpt-lands-2024-newembeds/chromadb", settings=Settings(allow_reset=True))

## Indexing

In this step, we will index the documents in our vector database. This will allow us to retrieve the most relevant documents when we ask a question.

We will use ChromaDB as our vector database and 'text-embedding-3-small' from OpenAI as our embedding model.


#### Fetch and process saved documents

First we need to fetch the documents we saved earlier.

Then we will process the documents in order to add them to our vector database.
The `SimpleDirectoryReader` fetches each section of the markdown file
Then each section is split in to smaller chunks of text and each chunk is embedded using the OpenAI API.


In [16]:
documents = SimpleDirectoryReader('./data/docs_lands_test').load_data()

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=20)

document_data = []

for document in documents:
    chunks = text_splitter.split_text(document.text)
    for idx, chunk in enumerate(chunks):
        embedding = openai_client.embeddings.create(
            input=chunk, model="text-embedding-3-small")#"text-embedding-ada-002")
        document_data.append({
            "id": f"{document.id_}-{idx}",
            "text": chunk,
            "metadata": document.metadata,
            "embedding": embedding.data[0].embedding
        })

In [17]:
print("embeddings dim", len(document_data[0]['embedding']))
print(document_data[0])

embeddings dim 1536
{'id': 'b7b42a4b-f80b-4462-8057-68baa3267065-0', 'text': 'LANDSFORSØGENE  2024\nForsøg og undersøgelser i\nDansk Landbrugsrådgivning\nSamlet og udarbejdet af\nSEGES Innovation P/S, Planter & Miljø\nved chefkonsulent Jon Birger Pedersen\nAktiviteterne er blandt andet støttet af:', 'metadata': {'page_label': '1', 'file_name': 'Landsforsøgene 2024.pdf', 'file_path': 'c:\\Users\\jach\\projekter\\ADVANCED-RAG-EXAMPLES\\data\\docs_lands_test\\Landsforsøgene 2024.pdf', 'file_type': 'application/pdf', 'file_size': 10628392, 'creation_date': '2024-12-19', 'last_modified_date': '2024-12-19'}, 'embedding': [0.0023039341904222965, 0.00349593092687428, 0.05882889777421951, 0.009601208381354809, -0.05897122621536255, 0.02993631921708584, -0.02157454937696457, -0.010763553902506828, 0.013260223902761936, -0.0036738410126417875, 0.046564970165491104, -0.003973322920501232, -0.004922175779938698, -0.028275825083255768, 0.01132100448012352, 0.006084521301090717, -0.024788789451122284

#### Add documents to ChromaDB


In [18]:
documents = [doc["text"] for doc in document_data]
embeddings = [doc["embedding"] for doc in document_data]
metadatas = [doc["metadata"] for doc in document_data]
ids = [doc["id"] for doc in document_data]

In [19]:
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="lands2024new", metadata={"hnsw:space": "cosine"}, embedding_function=openai_ef)

In [20]:
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=ids)



In [21]:
import chromadb
from chromadb.config import Settings

# Initialize the PersistentClient again with the same path
chroma_client_load = chromadb.PersistentClient(
    path="./data/seges-gpt-lands-2024-newembeds/chromadb",
    settings=Settings(allow_reset=True)
)

# Get the existing collection by name
collection_load = chroma_client_load.get_collection(name="lands2024new", embedding_function=openai_ef)

## Retrieval

In this step, we will retrieve the most relevant documents to a given question. We will use the vector database to retrieve the most similar documents to the question.

In order to do this we will use the `text-embedding-3-small` model (**the same model used to index the documents**) from OpenAI to embed the question and then use the vector database to retrieve the most similar documents.

We will retrieve the top 5 documents based on the _cosine similarity_ between the question and the documents. Other similarity metrics can be used as well like squared L2 or inner product.

Change `cosine` to `l2` or `ip` when creating the collection above to try these out.


In [ ]:
queries = [#"Hvad står P, L, A, C og E for i PLACE-akronymet af John Ziman",
         "hvilken brødhvedesort til høst i 2025?",
         ]

query = queries[-1]


'hvilken brødhvedesort til høst i 2025?'

In [11]:
collection_load

Collection(name=vtdat)

In [12]:
result = collection_load.query(query_texts=[query], n_results=5)
context = result["documents"][0]
#display(Markdown(f"------------\n\n{"\n\n------------\n\n".join(context)}"))

formatted_text = "\n\n------------\n\n".join(context)

# Display the formatted markdown
display(Markdown(f"{formatted_text}"))

221 KUL TURTEKNIK OG JORD  AFPRØVNING  AF FREMTIDENS   BÆREDYGTIGE  DYRKNINGS SYSTEMER
rødderne i minirhizotron-rør i Slagelse og Holstebro samt 
infiltrationsmålinger på alle lokaliteter. Disse data vil bli -
ve offentligt i en Landbrugsinfo-artikel i starten af 2025.
I forsøgene tilstræbes det at behandle de forskellige 
forsøgsled ud fra et mål om at optimere alle dyrknings-
systemerne. Såtid og udsædsmængde tilpasses jordens tjenlighed i de forskellige systemer, ligesom gødsknings- 
og planteværnsstrategi tilpasses aktuelle behov.
I 2024 er de tre storskalaforsøg videreført på fjerde år. 
Forsøgene evaluerer som nævnt dyrkningssystemerne: 
traditionel pløjning, reduceret jordbearbejdning og CA. I første forsøgsår var afgrøden hestebønner, efterfulgt af 
vinterhvede med efterafgrøde i 2022 og vårbyg i 2023. I 
efteråret 2023 blev der sået vinterraps på alle lokaliteter. På grund af dårlig fremspiring ved Slagelse, blev der om-
sået med vinterhvede. Ligeledes blev rapsen i Birkelse 
nedvisnet i foråret 2024 på grund af dårlig plantevækst. Her blev i stedet sået havre. Omsåningen skyldes de me -get våde forhold (se foto). Den dårlige plantevækst har 
ikke været påvirket af dyrkningssystemet.

------------

Her er det forsøget ved Esbjerg d. 1. maj 2024. 
Forsøget blev kasseret.FOTO: LEIF HAGELSKJÆR, SEGES INNOVATION
STRATEGI
Vælg altid en vinterhvedesort eller en 
sortsblanding, der:
 >har givet et stort udbytte gennem flere års for -
søg
 >har en god overvintringsevne
 >har en god stråstivhed, så den kan dyrkes uden 
vækstregulering
 >har en effektiv resistens over for følgende syg -
domme (i prioriteret rækkefølge):
 – Septoria
 – gulrust
 – meldug
 – brunrust.
En satsning på deciderede brødhvedesorter er kun aktuel, hvis der er rimelig sikkerhed for afsætning til en passende merpris.

------------

Vårhvedesorter, der har dækket over 1,0 procent af 
udsædsproduktionen i 2024. T abellen viser sorternes andel af udsædsproduktionen i procent
Høstår 2020 2021 2022 2023 2024
T on i alt 3.347 4.704 4.768 2.621 2.777
Kapitol 2 25 33 52 33
Bravens 3 14 19
KWS Fixum 10
KWS Sharki 3 8 9
Sonett 5 6 5 11 8
Dacke 2 5 3 1 7
Thorus 21 27 28 6 4
KWS Barolum 4
KWS T alisker 1 2 6 4 4
Andre sorter 70 36 19 4 2

------------

Det er bemærkelsesværdigt, at vinterbyg har klaret sig 
godt i et klimamæssigt vanskeligt år.
Der har været anlagt ti forsøg til høst 2024, hvoraf seks 
har givet brugbare resultater. T o af de kasserede forsøg 
har været vandlidende. I et forsøg har plantebestanden været for ringe og uens, og i det sidste forsøg har der væ -
ret en uensartet plantebestand på langs af parcellerne i halvdelen af forsøget. I tabel 2 er resultaterne af lands -
forsøgene opdelt på tre forsøg på Øerne og tre i Jylland. 
Udbytteniveauet er 21,9 hkg pr. ha højere på Øerne end 
i Jylland, og udbyttet varierer fra 74,6 hkg pr. ha i Nord-jylland til 117,0 hkg pr. ha på Lolland i målesortsblandin -
gen. Det er en stor variation, som typisk ses i klimamæs-sigt udfordrende år, enten med tørke eller med megen nedbør. De mest dyrkede sorter til høst 2024 har været de toradede LG Globetrotter, Bordeaux, Alaska og Or -
cade. LG Globetrotter og Bordeaux har ikke været med 
i Landsforsøgene i 2024, mens Alaska og Orcade har gi-
vet forholdstal 98 og 97. Desuden har sortsblandinger udgjort 18 procent af den solgte udsæd. Sortsblandin -
gerne har ikke været med i Landsforsøgene, men en af blandingerne har været med i de supplerende forsøg og giver forholdstal 99.
Y derst til højre i tabel 2 er indholdet af råprotein  og rum -
vægt angivet i gennemsnit af de seks forsøg. Proteinind -TABEL 1.

------------

125 BÆLGSÆD  HESTEBØNNER , SORTER
resultater sammenholdes med resultaterne i tabel 1 og 
2, kan man sikre sig et godt overblik over sorternes ud -
byttestabilitet og øvrige dyrkningsegenskaber.Den producerede mængde certificeret udsæd af mar -
kært i foråret 2024 fremgår af tabel 4 sammen med for -
delingen på de samme sorter fra 2020. Produktionen af ærteudsæd har i 2024 været cirka 20 procent lavere end 
i 2023, og kan skyldes de sene etableringsmuligheder i 
2024. Javlo anvendes næsten alene i byg/ærteblandin -
ger eller til ærtehelsæd. Den mest dyrkede sort til mo -
denhed er for niende år i træk Ingrid, der har udgjort 26 procent af den producerede udsæd.
Vintermarkærter
Der har i efteråret 2023 været lavet en plan for afprøv -
ning af sorter af vintermarkærter i 2024 (010502424- 
vintermarkærter) på to milde lokaliteter på Lolland og 
i Østjylland. Grundet den megen nedbør blev forsøget i Østjylland ikke anlagt i efteråret 2023, og forsøget på 
Lolland blev ikke høstet i 2024 på grund af kraftige fug -
leskader. Forsøgene gentages igen i 2024/2025 på andre 
lokaliteter.
Hestebønner, sorter
 >MARIAN DAMSGAARD THORSTED OG  
JON BIRGER PEDERSEN, SEGES INNOVATION
Vårhestebønner
Loki, der deltager i Landsforsøgene® for første gang i 
2024, giver det højeste udbytte i årets forsøg med heste-
bønner svarende til forholdstal 107.

## Generation

In this step, we will generate an answer to the question using the retrieved documents as context. We will use the OpenAI API to generate the answer.


In [13]:
prompt = PromptTemplate("""You are a helpful assistant that answers questions about the US tv show known as "The Office" using provided context. 

Question: {query}

Context: 

-----------------------------------
{context}

-----------------------------------

""")

prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the course material from "Philosophy of Computer Science (VtDat)" using provided context.
    Context information is below.
    ---------------------
    {context}
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the Danish language.
    Query: {query}
    Answer: 
    """,
)


message = prompt.format(query=query, context="\n\n".join(context))
display(Markdown(f"{message}"))

You are a helpful assistant that answers questions about the course material from "Philosophy of Computer Science (VtDat)" using provided context.
    Context information is below.
    ---------------------
    221 KUL TURTEKNIK OG JORD  AFPRØVNING  AF FREMTIDENS   BÆREDYGTIGE  DYRKNINGS SYSTEMER
rødderne i minirhizotron-rør i Slagelse og Holstebro samt 
infiltrationsmålinger på alle lokaliteter. Disse data vil bli -
ve offentligt i en Landbrugsinfo-artikel i starten af 2025.
I forsøgene tilstræbes det at behandle de forskellige 
forsøgsled ud fra et mål om at optimere alle dyrknings-
systemerne. Såtid og udsædsmængde tilpasses jordens tjenlighed i de forskellige systemer, ligesom gødsknings- 
og planteværnsstrategi tilpasses aktuelle behov.
I 2024 er de tre storskalaforsøg videreført på fjerde år. 
Forsøgene evaluerer som nævnt dyrkningssystemerne: 
traditionel pløjning, reduceret jordbearbejdning og CA. I første forsøgsår var afgrøden hestebønner, efterfulgt af 
vinterhvede med efterafgrøde i 2022 og vårbyg i 2023. I 
efteråret 2023 blev der sået vinterraps på alle lokaliteter. På grund af dårlig fremspiring ved Slagelse, blev der om-
sået med vinterhvede. Ligeledes blev rapsen i Birkelse 
nedvisnet i foråret 2024 på grund af dårlig plantevækst. Her blev i stedet sået havre. Omsåningen skyldes de me -get våde forhold (se foto). Den dårlige plantevækst har 
ikke været påvirket af dyrkningssystemet.

Her er det forsøget ved Esbjerg d. 1. maj 2024. 
Forsøget blev kasseret.FOTO: LEIF HAGELSKJÆR, SEGES INNOVATION
STRATEGI
Vælg altid en vinterhvedesort eller en 
sortsblanding, der:
 >har givet et stort udbytte gennem flere års for -
søg
 >har en god overvintringsevne
 >har en god stråstivhed, så den kan dyrkes uden 
vækstregulering
 >har en effektiv resistens over for følgende syg -
domme (i prioriteret rækkefølge):
 – Septoria
 – gulrust
 – meldug
 – brunrust.
En satsning på deciderede brødhvedesorter er kun aktuel, hvis der er rimelig sikkerhed for afsætning til en passende merpris.

Vårhvedesorter, der har dækket over 1,0 procent af 
udsædsproduktionen i 2024. T abellen viser sorternes andel af udsædsproduktionen i procent
Høstår 2020 2021 2022 2023 2024
T on i alt 3.347 4.704 4.768 2.621 2.777
Kapitol 2 25 33 52 33
Bravens 3 14 19
KWS Fixum 10
KWS Sharki 3 8 9
Sonett 5 6 5 11 8
Dacke 2 5 3 1 7
Thorus 21 27 28 6 4
KWS Barolum 4
KWS T alisker 1 2 6 4 4
Andre sorter 70 36 19 4 2

Det er bemærkelsesværdigt, at vinterbyg har klaret sig 
godt i et klimamæssigt vanskeligt år.
Der har været anlagt ti forsøg til høst 2024, hvoraf seks 
har givet brugbare resultater. T o af de kasserede forsøg 
har været vandlidende. I et forsøg har plantebestanden været for ringe og uens, og i det sidste forsøg har der væ -
ret en uensartet plantebestand på langs af parcellerne i halvdelen af forsøget. I tabel 2 er resultaterne af lands -
forsøgene opdelt på tre forsøg på Øerne og tre i Jylland. 
Udbytteniveauet er 21,9 hkg pr. ha højere på Øerne end 
i Jylland, og udbyttet varierer fra 74,6 hkg pr. ha i Nord-jylland til 117,0 hkg pr. ha på Lolland i målesortsblandin -
gen. Det er en stor variation, som typisk ses i klimamæs-sigt udfordrende år, enten med tørke eller med megen nedbør. De mest dyrkede sorter til høst 2024 har været de toradede LG Globetrotter, Bordeaux, Alaska og Or -
cade. LG Globetrotter og Bordeaux har ikke været med 
i Landsforsøgene i 2024, mens Alaska og Orcade har gi-
vet forholdstal 98 og 97. Desuden har sortsblandinger udgjort 18 procent af den solgte udsæd. Sortsblandin -
gerne har ikke været med i Landsforsøgene, men en af blandingerne har været med i de supplerende forsøg og giver forholdstal 99.
Y derst til højre i tabel 2 er indholdet af råprotein  og rum -
vægt angivet i gennemsnit af de seks forsøg. Proteinind -TABEL 1.

125 BÆLGSÆD  HESTEBØNNER , SORTER
resultater sammenholdes med resultaterne i tabel 1 og 
2, kan man sikre sig et godt overblik over sorternes ud -
byttestabilitet og øvrige dyrkningsegenskaber.Den producerede mængde certificeret udsæd af mar -
kært i foråret 2024 fremgår af tabel 4 sammen med for -
delingen på de samme sorter fra 2020. Produktionen af ærteudsæd har i 2024 været cirka 20 procent lavere end 
i 2023, og kan skyldes de sene etableringsmuligheder i 
2024. Javlo anvendes næsten alene i byg/ærteblandin -
ger eller til ærtehelsæd. Den mest dyrkede sort til mo -
denhed er for niende år i træk Ingrid, der har udgjort 26 procent af den producerede udsæd.
Vintermarkærter
Der har i efteråret 2023 været lavet en plan for afprøv -
ning af sorter af vintermarkærter i 2024 (010502424- 
vintermarkærter) på to milde lokaliteter på Lolland og 
i Østjylland. Grundet den megen nedbør blev forsøget i Østjylland ikke anlagt i efteråret 2023, og forsøget på 
Lolland blev ikke høstet i 2024 på grund af kraftige fug -
leskader. Forsøgene gentages igen i 2024/2025 på andre 
lokaliteter.
Hestebønner, sorter
 >MARIAN DAMSGAARD THORSTED OG  
JON BIRGER PEDERSEN, SEGES INNOVATION
Vårhestebønner
Loki, der deltager i Landsforsøgene® for første gang i 
2024, giver det højeste udbytte i årets forsøg med heste-
bønner svarende til forholdstal 107.
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the Danish language.
    Query: hvilken brødhvedesort til høst i 2025?
    Answer: 
    

In [14]:
stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": query}],
    model="gpt-4",
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

Som kunstig intelligens har jeg ingen iboende evne til at forudsige fremtidige begivenheder eller tendenser inden for landbrug eller kornsorter i 2025. Det anbefales at henvende sig til en agronom eller en anden ekspert inden for området for at få en præcis og opdateret anbefaling baseret på de seneste forskning og trends inden for branchen.

In [15]:
stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": message}],
    model="gpt-4",
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

Konteksten indeholder ikke specifik information om, hvilken brødhvedesort der skal høstes i 2025.

## SlideGPT

In [24]:
queries = [
    #"Hvad står P, L, A, C og E for i PLACE-akronymet af John Ziman"
    #"Redegør for hovedtrækkene i sagen om COMPAS-algoritmen som den beskrives i Angwin et al. (2016)."
    #"Hvad er verifikation og falsifikation?"
    #"Redegør for hovedtrækkene i sagen om COMPAS-algoritmen som den beskrives i Angwin et al. (2016)."
    #"2. COMPAS-algoritmen inddrager ikke direkte etnicitet, men den inddrager faktorer, der kan være korreleret med etnicitet (Angwin et al., 2016, 3–4). Har man som modelkonstruktør et ansvar for at undlade at inddrage faktorer, der kan korreleres med etnicitet eller andre følsomme forhold? Kan man det? Hvilke etiske overvejelser bør man have. Inddrag synspunkter fra nytte og pligtetik (kategoriske og praktiske imperativ), og Nagels synsteseetik. Hvorfor har modelkonstruktøren et etisk ansvar?"
    #"Forklar begreberne group fairness og individuel fairness fra Friedler et al. (2021) og forklar hvorfor en algoritme (normalt) ikke kan udvise gruppe- og individuel fairness samtidig."
    "Analyser hvordan disse fairness-begreber (group fairness og individuel fairness) er på spil i COMPAS-casen"
]

query = queries[-1]

result = collection.query(query_texts=[query], n_results=10)
context = result["documents"][0]
#display(Markdown(f"------------\n\n{"\n\n------------\n\n".join(context)}"))

formatted_text = "\n\n------------\n\n".join(context)

# Display the formatted markdown
display(Markdown(f"{formatted_text}"))



prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the course material from "Philosophy of Computer Science (VtDat)" using provided context.
    Specifically, you will provide your answer such that it is generated as PowerPoint slide(s) content with 3 PowerPoint slides. If helpful, please provide the answer using bullet points.
    Context information is below.
    ---------------------
    {context}
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the Danish language.
    Query: {query}
    Answer: 
    """,
)

message = prompt.format(query=query, context="\n\n".join(context))
display(Markdown(f"{message}"))

stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": message}],
    model="gpt-4",
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

Slide 1:
Title: Analyse af Fairness i COMPAS-casen
- Introduktion til COMPAS
   - COMPAS-algoritmen bruges i USA's retssystem til bedømmelse af kriminelles risiko for recidivisme.
   - Et review af denne algoritme viste en systematisk skævhed i dens fejl. 
   - Personer af farvet hud, der ikke begår ny kriminalitet, har langt højere chance for fejlagtigt at få en høj risikoscore end hvide. Omvendt gælder det for hvide for fejlagtigt at få en lav score.
   
Slide 2:
Title: Gruppe Fairness og COMPAS
- Gruppe Fairness involverer at behandle grupper af mennesker ligeligt. 
- I COMPAS-casen:
   - Det forekommer at der er en skævhed mod farvede personer. 
   - Mulighed for gruppe-diskrimination, hvor folk med samme etniske baggrund behandles anderledes end andre grupper.
   
Slide 3:
Title: Individuel Fairness og COMPAS
- Individuel fairness handler om at behandle hver enkelt individ retfærdigt.
- I COMPAS-casen:
   - COMPAS-algoritmen kan potentielt behandle individuelle farvede personer uretfærdigt baseret på deres hudfarve. 
   - Potentialet for diskrimination på individuelt niveau selv hvis gruppefairness er opfyldt.
- Afsluttende tanker: Vi skal være omhyggelige med de data, vi bruger til at træne algoritmer, og være opmærksomme på potentialerne for skjulte forudsætninger og diskriminationer.

In [23]:
display(Markdown(f"{message}"))

You are a helpful assistant that answers questions about the course material from "Philosophy of Computer Science (VtDat)" using provided context.
    Specifically, you will provide your answer such that it is generated as PowerPoint slide(s) content with 3 PowerPoint slides. If helpful, please provide the answer using bullet points.
    Context information is below.
    ---------------------
    F or det første er de etiske argumenter
for at bruge prædiktive algoritmer typisk utilitaristiske; den grundlæggende påstand er, at
en algoritme som COMP AS på den eller anden måde øger den samlede nytte i samfundet og
det er på den baggrund algoritmen skal vurderes (der kan selvfølgelig også være ikke-etiske
begrundelser som kommercielle interesser eller andet). 620
F or andet skal være klar over, at prædiktive algoritmer altid diskriminerer, i den forstand
at din fremtidige adfærd estimeres på baggrund af den fortidige adfærd af den gruppe, som
du anses for at tilhøre. Hvis man fx vil estimere hvor længe en 25 årig, ikke-rygende, køben-
havnsk studerende lever, vil man se på, hvor længe folk i en referenceklasse af personer, der
minder om vores 25-årige studerende i gennemsnit har levet. Det er ganske simpelt sådan, 625
statistiske forudsigelser fungerer (i det mindste i den frekventistisk tolkning af sandsynlig-
hed, som er mest naturlig i denne sammenhæng). Spørgsmålet er altså ikke om algoritmerne
må diskriminere, men i højere grad hvilke faktorer, vi vil tillade, at der indgår i reference-
klassen. Hvilke træk ved dig, kan man det etisk set forsvare at diskriminere på baggrund
af, når du skal have en livsforsikring, ansøger om et job eller søger ind på en uddannelse? 630
F or at svare på det spørgsmål bliver vi for det tredje nødt til at inddrage begrebet ret-
færdighed.

også et socialt gode, idet den grundlæggende autonomi og integritet, beskyttelse af privat-
livet muliggør, ifølge Johnson er en afgørende forudsætning for at have et demokrati. Et
demokrati kan kun fungere, hvis der er plads til at tænke anderledes og mulighed for at gøre 560
og afprøve nye ting, som de siddende magthaverne måske ikke billiger. Uden en privatsfære,
hvor man kan tage selvstændig stilling til og indgå med andre borgere i en kritisk dialog om
magthavernes beslutninger kan man ikke have et demokrati. Hvis det argument er korrekt,
er spørgsmålet om privatliv vs. sikkerhed ikke blot et spørgsmål om individuelle rettigheder
mod almenvellets velfærd, men også et spørgsmål om forskellige former for generel velfærd. 565
Sat på spidsen kan man sige, at valgte også står mellem den sikkerhed man kan få med
totalitær overvågning og de fordele, et oplyst, inddragende demokratisk styre giver. Man
kan i længden ikke have begge dele.
10.7 Bias og algoritmisk transparens
Brugen af algoritmer giver også en anden etisk konflikt, nemlig en konflikt mellem effek- 570
tivitet og retfærdighed. Det har i århundrede været helt almindeligt at bruge datadrevne
sandsynlighedsberegninger til at støtte beslutninger, der går ud på at vurdere en form for
risiko. Hvis man fx tegner en ulykkesforsikring, vil forsikringsselskabet bruge deres viden
om, hvor hyppigt folk, der på relevante træk ligner dig, kommer til skade, til at fastsætte
forsikringspræmien.

En
algoritme som COMP AS, der har en større tilbøjelighed til at holde farvede i fængsel end
ikke-farvede, vil bidrage til fastholde denne strukturelle ulighed. En datadreven algoritme
afspejler virkeligheden, som den er, men hvis virkeligheden er racistisk, vil algoritmen også
blive det, og hvis algoritmen bliver brugt uden forbehold, kan den bidrage til at fastholde 685
historisk betingede diskriminerende strukturer. Der er derimod ikke noget, der tyder på, at
mænds kortere levealder er et udtryk for strukturel diskrimination. Så med andre ord er
det vigtigt, at se på den kontekst, en algoritme skal indgå i, når man overvejer, om den er
retfærdig.
Pandoras black box 690
I en traditionel statistisk model vil man have nogenlunde styr på de variable, modellen
inddrager. Det er dog værd at bemærke, at tingene selv med traditionelle algoritmer ik-
ke altid er så klare; COMP AS-algoritmen inddrager således ikke direkte etnicitet som en
variabel. Den inddrager til gengæld flere variable, der er tæt korrelerede med etnicitet, og
dermed indgår etnicitet indirekte i algoritmen. Det gør naturligvis tingene mere besværlige, 695
16Henrik Kragh Sørensen og Mikkel Willum Johansen (apr. 2022). „Invitation til de datalogiske fags
videnskabsteori“ . Lærebog til brug for undervisning ved Institut for Naturfagenes Didaktik, Københavns
Universitet. Under udarbejdelse.
Kapitel 10, version 151 (2021-06-05).

Algoritmen til forudsigelse af frafald inddrog således direkte elevernes etnicitet
som en faktor i beregningen, men hvad nu hvis algoritmen havde en bias så den systematisk
gav elever med en etnicitet en højre risikovurdering end elever med en anden?
Spørgsmålet om diskrimination i algoritmer er navnligt blevet diskuteret i forbindelse 595
med den såkaldte COMP AS-algoritme, der bruges i det amerikanske retsvæsen til vurdering
af kriminelles tilbagefaldsrisiko. En gennemgang af algoritmen viste imidlertid en systema-
tisk skævhed i de fejl, algoritmen lavede. Således havde farvede, der ikke senere begik ny
kriminalitet, en langt højere chance for fejlagtigt at få en høj risikoscore end hvide, og hvide
havde omvendt en højere chance for fejlagtigt at få en lav score end farvede (Angwin m.fl., 600
2016 ).
Hvis man imidlertid tager udgangspunkt i de to kategorier høj- og lav-risiko, og under-
14Henrik Kragh Sørensen og Mikkel Willum Johansen (apr. 2022). „Invitation til de datalogiske fags
videnskabsteori“ . Lærebog til brug for undervisning ved Institut for Naturfagenes Didaktik, Københavns
Universitet. Under udarbejdelse.
Kapitel 10, version 151 (2021-06-05).

Algoritmisk bias er et nært forbun-
det fænomen. Som vi så ovenfor havde Flu Trends svært ved at skel-ne in ﬂ uenza- og vintersæsonen, for-
di de to fænomener forekom samti-digt i det oprindelige træningssæt. Det var et ret uskyldigt eksempel, men hvad nu hvis de to fænomener, der var blevet sammenblandet, var noget mere følsomt som etnicitet og kriminalitet? I navnlig det ameri-kanske retssystem støtter man sig i stigende grad til statistisk trænede algoritmer, når man skal afgøre, hvorvidt fanger skal prøveløslades. Fangerne skal udfylde et spørge-skema, og på baggrund af, hvordan det er gået fanger, der tidligere har udfyldt samme skema, kan man udregne en score for, hvor sandsyn-ligt det er, at en person vil begå ny kriminalitet. Metoden er imidlertid blevet beskyldt for konsekvent at give sorte en højre kriminalitets-score end folk af andre hudfarver. Man må ikke dømme folk alene ud fra deres hudfarve – det er racisme – men noget tyder altså på, at al-goritmen i dette tilfælde til dels har lært at gætte, om folk er sorte eller ej, præcis som Flu Trends til dels havde lært at gætte på, om vinteren nærmede sig. 
Vi risikerer naturligvis altid at 
blive mødt af fordomme, men når fordommene bygges ind i model-ler, der fremstår som neutrale og objektive, kan de være sværere at adressere. Når man bruger big data er der derfor ikke bare gode metodologiske grunde til at være opmærksom på sammenblandin-gen mellem over ﬂ adisk relaterede 
fænomener. Der kan også være gode etiske grunde til det.

søger, hvor god algoritmen er til at komme med korrekte forudsigelser, er der paradoksalt
nok ingen forskel på farvede og hvide; blandt de, der blev bedømt som højrisiko, var pro-
centdelen af farvede, der var blevet fejlplaceret, nogenlunde lige så stor som procentdelen 605
af hvide, og tilsvarende for de, der blev vurderet som højrisiko. Det betyder med andre
ord, at risikoscoren er nogenlunde lige pålidelig, uanset hvilken etnicitet den person, den
bliver lavet for, har. Y dermere har det vist sig, at man ikke kan have lighed for begge mål
samtidig. Enten vil algoritmen som nu være lige pålidelig i sine forudsigelser uanset etnici-
tet, men til gengæld have en systematisk skævhed i forhold til enkeltpersoners risiko for at 610
blive fejlplaceret, eller også kan man gøre risikoen for at blive fejlplaceret lige stor uanset
etnicitet, hvilket til gengæld betyder, at algoritmen vil være mindre pålidelig for den ene
etniske gruppe end for den anden (for en stringent matematiske gennemgang, se Kleinberg,
Mullainathan og Raghavan, 2017 ). Det er selvfølgelig en interessant etisk udfordring!
Når man skal diskutere hvorvidt brugen af prædiktive algoritmer kan forsvares etisk er 615
der er en række ting, det kan være godt at få på plads.

Det er ret ukontroversielt. Det er straks mere kontroversielt, at man 575
bl.a. i det amerikanske retsvæsen siden starten af den 20. århundrede har brugt lignende
beregninger til at vurdere kriminelles risiko for at begå ny kriminalitet – og at den type
beregninger er blevet brugt i vurderingen strafudmåling, prøveløsladelser mv. (Carlson, A.
(2017). The Need for T ransparency in the Age of Predictive Sentencing Algorithms. Iowa
Law Review, 103(1), 303–329.). I de sidste årtier har lettere adgang til store datamængder 580
og fremkomsten af diverse machine learning teknikker gjort det lettere at udvikle den type
systemer og diverse former for prædiktive algoritmer bruges i dag på en lang række områder
fra reklamer til sortering af jobansøgninger.
Det er dog ikke helt uproblematisk at bruge prædiktive algoritmer. Som vi så tidligere, er
der en række epistemiske problemer forbundet med machine learning. Der er typisk en meget 585
direkte sammenhæng mellem epistemiske og etiske overvejelser; hvis de etiske overvejelser
inddrager utilititaristiske komponenter er det i høj grad væsentligt at forstå, hvor godt en
given algoritme i praksis virker. Ville den etiske vurdering af algoritmen til forudsigelse af
frafald i gymnasiet fx falde anderledes ud, hvis systemet kun havde en træfsikkerhed på
67%? Der er imidlertid også et anden og vanskeligere etisk aspekt specielt hvis man bruger 590
algoritmiske forudsigelser til at træffe afgørelser, der har betydning for enkeltindividers
muligheder.

at man på den måde kan komme til at bruge variable, man ikke direkte har indbygget i sin
algoritme.
Det problem bliver imidlertid kraftigt forstærket, hvis man træner en algoritme med
machine learning. Her har vi styr på de data, algoritmen er trænet med, og vi kan opstille
forskellige mål for, hvor godt den virker, men resten er en black box; vi aner reelt ikke 700
hvorfor den virker, og som vi berørte ovenfor, kan machine learning algoritmer med lethed
diskriminere langs variable, der er af en helt anden type den de data, vi har trænet algo-
ritmen med. Hvis vi vil sikre, at de algoritmer, vi udvikler, ikke diskriminerer på en etisk
kritisabel måde, bliver vi nødt til at gøre dem transparente. Det er imidlertid ikke altid let,
at gøre en blackbox transparent! 705
10.8 Systemisk etik
Efter at have set på de normative teorier vil vi nu vende tilbage til den deskriptive etik og
beskrive et interessant og i høj grad overset aspekt af etikken, nemlig det tilsyneladende
empiriske faktum, at vores etiske beslutninger kan påvirkes af selv små ændringer i vores
miljø. Som et enkelt eksempel testede John M. Darley (1938–2018) og Daniel Batson 710
teologistuderende i et eksperimentelt paradigme, hvor en forsøgsperson (typisk en stude-
rende) bliver bedt om at forberede et kort oplæg om den barmhjertige samaritaner, som
er en lignelse fra Det Nye T estamente, der i korte træk går ud på, at man ubetinget skal
hjælpe andre i nød, også selv om det er en vildt fremmet af et andet folk (Darley og Bat-
son, 1973 ).

Og hvad gør man så? Er
det mest retfærdige at kønsdiskriminere i dette tilfælde (og lade de fornuftige mænd betale 665
prisen) eller er det mest retfærdigt at undlade at kønsdiskriminere (og lade kvinderne betale
prisen)?
Det bringer os tilbage til COMP AS-algoritmen. I den population, hvor algoritmen er
blevet testet, har farvede fanger reelt en større risiko for at begå ny kriminalitet. Når
algoritmen diskriminerer på baggrund af etnicitet er det derfor blot et udtryk for, at den 670
har identificeret en informationsbærende dimension i datasættet. Og lige som ovenfor vil vi
have valget imellem at forhindre algoritmen i at bruge informationen, hvorved den vil blive
mindre effektiv i den forstand at dens overordnede fejlrate vil stige, eller at tillade den at
bruge informationen, hvorved den vil diskriminere enkeltindivider, således at farvede, der
ikke vil begå ny kriminalitet har en meget større risiko for at få en høj kriminalitetsscore 675
end ikke-farvede. Hvilket af de to scenarier forekommer dig mest retfærdigt, når du står
bag uvidenhedens slør?
Der er dog også væsentlige forskelle på de to cases. Specielt kan man argumentere for, at
den gennensnitligt høje kriminalitetsrate blandt farvede til dels skyldes strukturelle forhold
i det amerikanske samfund; pga. en generel diskrimination mod farvede har de sværere ved 680
at få uddannelser og job end ikke-farvede, og derfor vil de hyppigere ende i kriminalitet.

Begrebet retfærdighed befinder sig et sted imellem etik og politisk filosofi, idet
det angår mere generelle egenskaber samfund, som fordeling af goder, adgang til politisk
magt og det juridiske straffesystem. Præcis som det er tilfældet for etik findes der mange
forskellige retninger indenfor politisk filosofi, og mange forskellige ideer om, hvad retfærdig- 635
hed er. Det vil vi på ingen måde gå i dybden med her. Vi vil nøjes med at præsentere en
enkelt og meget operationel forestilling om retfærdighed, til dels med udgangspunkt i den
amerikanske filosof John Ra wls (1921–2002). Ra wls hævder (Rawls, 1972 ), at hvis sam-
fundet skal være retfærdigt, skal vi tage alle grundlæggende beslutninger om det indretning
og institutioner, mens vi befinder os bag et uvidenhedens slør , der skjuler, hvilken position 640
vi selv vil få i samfundet. Så man skal altså forestille sig, at man ikke ved, hvilket køn man
har, at man ikke ved, om man er rig eller fattig, at man ikke kender sin etnicitet mv., og
ud fra den position kan man beslutte, hvordan samfundet skal indrettes. I dette tilfælde:
Hvilke former for diskrimination vil vi tillade i prædiktive algoritmer? Det er let at forestille
sig (men det kan selvfølgelig diskuteres) at man i den situation vil vælge et samfund der 645
er uden kønsdiskrimination og ude diskrimination mode fx seksuelle og etniske minoriteter.
Så simpelt er det.
Og så alligevel ikke, for der kan være principielt forskellige former for diskrimination.
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the Danish language.
    Query: Forklar begreberne group fairness og individuel fairness fra Friedler et al. (2021) og forklar hvorfor en algoritme (normalt) ikke kan udvise gruppe- og individuel fairness samtidig.
    Answer: 
    

## Normal example using LlamaIndex

In this example, we will use LlamaIndex to abstract the indexing and retrieval steps. This shows how easily the same pipeline can be implemented using LlamaIndex.


In [ ]:

#%pip install llama-index-embeddings-azure-openai
#%pip install llama-index-llms-azure-openai

In [12]:
import chromadb
from chromadb import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# ChromaDB Vector Store
chroma_client = chromadb.PersistentClient(
    path="./data/baseline-rag-pdf-docs/chromadb", settings=Settings(allow_reset=True))
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="VtDat", metadata={"hnsw:space": "cosine"})
vector_store = ChromaVectorStore(chroma_collection=collection)


llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="gpt-4",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Define the ingestion pipeline to add documents to vector store
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=20),
        embed_model,
    ],
    vector_store=vector_store,
)

# Create index with the vector store and using the embedding model
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model)

In [13]:
# Fetch documents
documents = SimpleDirectoryReader('./data/docs').load_data()

# Run pipeline
pipeline.run(documents=documents)

print("Indexing complete")

Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 65 0 (offset 0)
Ignoring wrong pointing object 96 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)


Indexing complete


In [46]:
index.as_retriever()

In [1]:
import hmac
from openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
import streamlit as st
import chromadb
import chromadb.utils.embedding_functions as embedding_functions


client = AzureOpenAI(api_key=st.secrets["OPENAI_API_KEY"], 
                api_version="2024-05-01-preview", 
                azure_endpoint=st.secrets["AZURE_OPENAI_ENDPOINT"])

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=st.secrets["EMBEDDING_OPENAI_API_KEY"],
    model_name=st.secrets["EMBEDDING_OPENAI_DEPLOYMENT_NAME"],
    api_type="azure",
    api_version=st.secrets["EMBEDDING_OPENAI_API_VERSION"]
)

#embed_model = AzureOpenAIEmbedding(
#    model=st.secrets["EMBEDDING_OPENAI_DEPLOYMENT_NAME"],
#    deployment_name=st.secrets["EMBEDDING_OPENAI_DEPLOYMENT_NAME"],
#    api_key=st.secrets["EMBEDDING_OPENAI_API_KEY"],  
#    api_version=st.secrets["EMBEDDING_OPENAI_API_VERSION"], # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
#    azure_endpoint=st.secrets["EMBEDDING_AZURE_OPENAI_ENDPOINT"]
#)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     c:\Users\jach\AppData\Local\miniconda3\envs\advanced-
[nltk_data]     rag-env\Lib\site-
[nltk_data]     packages\llama_index\core\_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


2024-12-14 22:15:45.102 
  command:

    streamlit run c:\Users\jach\AppData\Local\miniconda3\envs\advanced-rag-env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [5]:
st.secrets["EMBEDDING_OPENAI_API_VERSION"]

'2024-05-01-preview'

In [2]:
openai_ef("hello")

AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}

In [21]:
from llama_index.core import PromptTemplate
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer

    
qa_prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the content of documents and provides detailed expert advice. 
    You must provide your answer in the Danish language.
    If the answer contains multiple steps or points, provide the answer in a bullet format.
    Below the answer, the source of the answer should be provided including file name and page number.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {query_str}
    Answer: 
    """,
)


class RAGQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: OpenAI
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        context_str = "\n\n".join([n.node.get_content(metadata_mode="all") for n in nodes])
        #context = qa_prompt.format(
        #    context_str=context_str, query_str=query_str)
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )
                    
        return str(response) + "\n\n-------------------------\n\nKontekst:\n\n" + context_str


synthesizer = get_response_synthesizer(response_mode="compact")
query_engine = RAGQueryEngine(
    retriever=index.as_retriever(),
    response_synthesizer=synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
)

TypeError: Can't instantiate abstract class OpenAI with abstract method _prepare_chat_with_tools

#### Create base QueryEngine from LlamaIndex


In [26]:
query_engine = index.as_query_engine(llm=llm, verbose=True)

#### Or alternatively, create a CustomQueryEngine


In [20]:
from llama_index.core import PromptTemplate
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer


class MyRetriever(BaseRetriever):
    def retrieve(self, query_str: str, max_documents: int = 10):
        # Get all documents relevant to the query
        all_documents = super().retrieve(query_str)

        # Return only the first `max_documents` documents
        return all_documents[:max_documents]


qa_prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the course material from "Philosophy of Computer Science (VtDat)" using provided context.
    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the Danish language.
    Query: {query_str}
    Answer: 
    """,
)


class RAGQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: MyRetriever #BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: OpenAI
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        #nodes = self.retriever.retrieve(query_str)#
        nodes = self.retriever.retrieve(query_str, max_documents=5)
        context_str = "\n\n".join([n.node.get_content() for n in nodes])
        print("Prompt:\n\n", qa_prompt.format(
            context_str=context_str, query_str=query_str))
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )

        return str(response)


synthesizer = get_response_synthesizer(response_mode="compact")
query_engine = RAGQueryEngine(
    retriever = index.as_retriever(),
    response_synthesizer=synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
)

TypeError: Can't instantiate abstract class MyRetriever with abstract method _retrieve

In [45]:
queries = [
    #"What are some of the made-up characters impersonated by Michael Scott? Give a oneliner description for each character.",
    #"Who is the character that is known for his 'That's what she said' jokes in The Office?",
    #"Which character loves cats?",
    #"A character has a heart attack in an episode of the show. Who is it?",
    #"What character lives proudly on a farm?",
    #"What happens to Kevin's Famous Chili when he brings it to the office?",
    #"Why did Michael Scott play the Savannah murder game?",
    #"How many marriages are in the show?",
    "Hvad dækker P, L, A, C og E over i PLACE-akronymet af John Ziman over?"
]

query = queries[-1]

response = query_engine.query(query)
display(Markdown(f"{response}"))

InvalidCollectionException: Collection e3406e00-da5e-41a3-b052-410e1e72d3a9 does not exist.

## Simplest RAG implementation using LlamaIndex


In [18]:
llm = AzureOpenAI(
    #model="gpt-4",
    #deployment_name="gpt-4",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Fetch documents
documents = SimpleDirectoryReader('./data/docs_lands_test').load_data()

from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

# build VectorStoreIndex that takes care of chunking documents
# and encoding chunks to embeddings for future retrieval
index = VectorStoreIndex.from_documents(documents=documents)#, 
                                        #api_key=os.environ['OPENAI_API_KEY'],
                                        #base_url=os.environ['AZURE_API_BASE'],
                                        #app_url=os.environ['AZURE_OPENAI_ENDPOINT'])
#index = VectorStoreIndex.from_documents(documents=documents, embed_model=embed_model, llm=llm, verbose=True)

# The QueryEngine class is equipped with the generator
# and facilitates the retrieval and generation steps
query_engine = index.as_query_engine()

# Use your Default RAG
response = query_engine.query(query)
display(Markdown(f"{response}"))

NameError: name 'AzureOpenAIEmbedding' is not defined

In [37]:

from ironpdf import *

# Load existing PDF document
pdf = PdfDocument.FromFile("C:/Users/jach/Documents/JacobsDocs/KU/VtDat/Tekster/Uge3/Kuhn_objektivitet.pdf")

# Extract text from specific page in the document
page_text = pdf.ExtractTextFromPage(1)


LicensingException: IronPDF must be licensed for development.
Please receive a free trial key instantly to your email by visiting:
https://ironpdf.com/licensing/?utm_source=product
   at IronPdf.License.kpbbda()
   at IronPdf.PdfDocument.axjqme(IEnumerable`1 ralkye, TextExtractionOrder ralkyf)
   at IronPdf.PdfDocument.ExtractTextFromPages(IEnumerable`1 PageIndices, TextExtractionOrder Order)
   at IronPdf.PdfDocument.ExtractTextFromPage(Int32 PageIndex, TextExtractionOrder Order)
   at IronPdf.PdfDocument.ExtractTextFromPage(Int32 PageIndex)
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodBaseInvoker.InvokeDirectByRefWithFewArgs(Object obj, Span`1 copyOfArgs, BindingFlags invokeAttr)